In [ ]:
import sys
sys.path.append("../")

# Get rid of annoying tf warning
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import bayesflow as beef
import pandas as pd

from experiments import RandomWalkDiffusionExperiment
from models import RandomWalkDiffusion
from priors import sample_scale

In [ ]:
# If set to False, existing results will be loaded
# Set to True if you want to re-run the experiments
TRAIN_NETWORKS = True
FIT_MODEL = True

## Neural Experiment

In [ ]:
model = RandomWalkDiffusion()
neural_experiment = RandomWalkDiffusionExperiment(model)

### Training

In [ ]:
if TRAIN_NETWORKS:
    history = neural_experiment.run(
        epochs=50, 
        iterations_per_epoch=1000, 
        batch_size=8
    )
else:
    history = neural_experiment.trainer.loss_history.get_plottable()

In [ ]:
f = beef.diagnostics.plot_losses(history)

## Fitting

In [ ]:
# read simulation data
simulation_output = pd.read_pickle('../data/static_dm_data_100.pkl')
sim_data = simulation_output['rt']
true_param = simulation_output['theta']

N_SIM = sim_data.shape[0]
NUM_STEPS = sim_data.shape[1]
N_SAMPLES = 4000

In [ ]:
if FIT_MODEL:
    neural_post_samples = np.zeros((N_SIM, NUM_STEPS, N_SAMPLES, 3))
    for i in range(N_SIM):
        neural_post_samples[i] = neural_experiment.amortizer.sample(
            {"summary_conditions": sim_data[:1]},
            N_SAMPLES
            )['local_samples']
    np.save('../data/neural_post_samples.npy', neural_post_samples)
else:
    neural_post_samples = np.load('../data/neural_post_samples.npy')

In [ ]:
neural_post_samples = neural_post_samples * model.local_prior_stds + model.local_prior_means

## Evaluation

In [ ]:
# read and prepare posteriors obtained with stan
stan_posteriors = np.load('../data/dynamic_stan_posteriors_100.npy')
# get posterior means
neural_post_means = neural_post_samples.mean(axis=2)
stan_post_means = stan_posteriors.mean(axis=2)
# calculate absolute error between posterior means and true parameter
neural_post_abs_error = np.empty((N_SIM, NUM_STEPS, 3))
stan_post_abs_error = np.empty((N_SIM, NUM_STEPS, 3))
for i in range(N_SIM):
    neural_post_abs_error[i] = np.abs(neural_post_means[i] - true_param[i])
    stan_post_abs_error[i] = np.abs(stan_post_means[i] - true_param[i])
# summarize absolute errors
neural_post_abs_error_mean = neural_post_abs_error.mean(axis=0)
neural_post_abs_error_std = neural_post_abs_error.std(axis=0)
stan_post_abs_error_mean = stan_post_abs_error.mean(axis=0)
stan_post_abs_error_std = stan_post_abs_error.std(axis=0)

In [ ]:
NEURAL_COLOR = '#852626'
COMPARISON_COLOR = '#133a76'
PARAMETER_LABELS = ['Drift rate', 'Threshold', 'Non-decision time']
PARAMETER_NAMES = [r'$v$', r'$a$', r'$\tau$']

In [ ]:
# sanity check
# which = 0
# f, axarr = plt.subplots(1, 3, figsize=(20, 5))
# for i, ax in enumerate(axarr.flat):
#     ax.plot(np.arange(NUM_STEPS), neural_post_means[which, :, i], color=NEURAL_COLOR)
#     ax.plot(np.arange(NUM_STEPS), np.repeat(true_param[which, i], NUM_STEPS), color=COMPARISON_COLOR)

In [ ]:
f, axarr = plt.subplots(1, 3, figsize=(20, 5))
for i, ax in enumerate(axarr.flat):
    ax.plot(np.arange(NUM_STEPS), neural_post_abs_error_mean[:, i], color=NEURAL_COLOR)
    ax.fill_between(
        np.arange(NUM_STEPS),
        neural_post_abs_error_mean[:, i] - neural_post_abs_error_std[:, i],
        neural_post_abs_error_mean[:, i] + neural_post_abs_error_std[:, i],
        color=NEURAL_COLOR, alpha=0.5
    )
    ax.plot(np.arange(NUM_STEPS), stan_post_abs_error_mean[:, i], color=COMPARISON_COLOR)
    ax.fill_between(
        np.arange(NUM_STEPS),
        stan_post_abs_error_mean[:, i] - stan_post_abs_error_std[:, i],
        stan_post_abs_error_mean[:, i] + stan_post_abs_error_std[:, i],
        color=COMPARISON_COLOR, alpha=0.5
    )

    ax.set_title(
        PARAMETER_LABELS[i] + ' ({})'.format(PARAMETER_NAMES[i]), fontsize=20)

sns.despine()